In [1]:
%load_ext autoreload
%autoreload 2
import os; import sys;sys.path.append("..")
import pandas as pd
import tqdm
import socceraction.spadl as spadl

In [ ]:
import urllib
import zipfile

url = "https://github.com/statsbomb/open-data/archive/master.zip"
urllib.request.urlretrieve(url, '../data/statsbomb-open-data.zip')

In [ ]:
with zipfile.ZipFile('../data/statsbomb-open-data.zip', 'r') as zipObj:
    zipObj.extractall("../data/statsbomb-json/")

In [27]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

data_url = "../data/statsbomb-json/open-data-master/data"
spadl.statsbombjson_to_statsbombh5(data_url,"../data/statsbomb.h5")

converting events files to ../data/statsbomb.h5: 100%|██████████| 258/258 [00:47<00:00,  5.19it/s]


In [28]:
pd.read_hdf("../data/statsbomb.h5","players")

,country_id,country_name,extra,jersey_number,player_id,player_name
0,22.0,Belgium,{},17.0,2954,Youri Tielemans
1,22.0,Belgium,{},5.0,3077,Jan Vertonghen
2,22.0,Belgium,{},7.0,3089,Kevin De Bruyne
3,22.0,Belgium,{},4.0,3101,Vincent Kompany
4,22.0,Belgium,{},15.0,3176,Thomas Meunier
5,22.0,Belgium,{},9.0,3289,Romelu Lukaku Menama
6,22.0,Belgium,{},8.0,3296,Marouane Fellaini-Bakkioui
7,22.0,Belgium,{},1.0,3509,Thibaut Courtois
8,22.0,Belgium,{},10.0,3621,Eden Hazard
9,22.0,Belgium,{},22.0,4831,Nacer Chadli


In [29]:
def convert_to_spadl(sbh5,spadlh5):
    matches = pd.read_hdf(sbh5,"matches")
    matches["game_id"] = matches.match_id
    games = matches
    games.to_hdf(spadlh5,"games")
    
    pd.read_hdf(sbh5,"players").to_hdf(spadlh5,"players")
    pd.read_hdf(sbh5,"teams").to_hdf(spadlh5,"players")
    pd.read_hdf(sbh5,"competitions").to_hdf(spadlh5,"competitions")
    
    for game_id in games.game_id:
        events = pd.read_hdf(sbh5,f"events/match_{game_id}")
        actions = convert_to_actions(events)
        break
        actions.to_hdf(spadlh5,f"actions/game_{game_id}")


sbh5 = "../data/statsbomb.h5"
spadlh5 = "../data/spadl-statsbomb.h5"
convert_to_spadl(sbh5,spadlh5)

In [31]:
import numpy as np
import pandas as pd


def convert_to_actions(events):
    actions = pd.DataFrame()
    
    events["extra"] = events["extra"].fillna({})
    events = events.fillna(0)
    
    actions["game_id"] = events.match_id
    actions["period_id"] = events.period
    
    actions["time_seconds"] = 60 * events.minute  - ((events.period == 2) * 45 *60) + events.second
    actions["timestamp"] = events.timestamp
    actions["team_id"] = events.team_id
    actions["player_id"] = events.player_id
    
    actions["start_x"] = events.location.apply(lambda x: x[0] if x else 1)
    actions["start_y"] = events.location.apply(lambda x: x[1] if x else 1)
    actions["start_x"] = ((actions["start_x"] -1) / 119) * 105
    actions["start_y"] = 68 - ((actions["start_y"] -1) / 79) * 68

    end_location = events[["location","extra"]].apply(get_end_location,axis=1)
    actions["end_x"] = end_location.apply(lambda x: x[0] if x else 1)
    actions["end_y"] = end_location.apply(lambda x: x[1] if x else 1)
    actions["end_x"] = ((actions["end_x"] -1) / 119) * 105
    actions["end_y"] = 68 - ((actions["end_y"] -1) / 79) * 68
    
    #print(events)
    return actions 


def get_end_location(q):
    start_location,extra = q
    if "pass" in extra and "end_location" in extra["pass"]:
        return extra["pass"]["end_location"]
    elif "shot" in extra and "end_location" in extra["shot"]:
        return extra["shot"]["end_location"]
    else:
        return start_location
    
def get_type_id(q):
    t = q,extra
    a = "non_action"
    if t == "Pass":
        a = "pass"
    elif t = "":
        pass


games = pd.read_hdf(spadlh5,"games")
#columns = set()
for game_id in games.game_id[250:251]:
    events = pd.read_hdf(sbh5,f"events/match_{game_id}")
    #columns = columns | set(events.columns)
    actions = convert_to_actions(events)
    
    #actions.to_hdf(spadlh5,f"actions/game_{game_id}")

#spadl.actiontypes
actions
#events

,game_id,period_id,time_seconds,timestamp,team_id,player_id,start_x,start_y,end_x,end_y
0,19767,1,0,00:00:00.000,974,0.0,0.000000,68.000000,0.000000,68.000000
1,19767,1,0,00:00:00.000,967,0.0,0.000000,68.000000,0.000000,68.000000
2,19767,1,0,00:00:00.000,967,0.0,0.000000,68.000000,0.000000,68.000000
3,19767,1,0,00:00:00.000,974,0.0,0.000000,68.000000,0.000000,68.000000
4,19767,1,0,00:00:00.242,967,18253.0,52.058824,34.430380,45.000000,36.151899
5,19767,1,1,00:00:01.165,967,15558.0,45.000000,36.151899,45.000000,36.151899
6,19767,1,1,00:00:01.325,967,15558.0,44.117647,36.151899,80.294118,68.000000
7,19767,1,15,00:00:15.714,974,15725.0,17.647059,0.000000,18.529412,30.987342
8,19767,1,17,00:00:17.750,974,10198.0,18.529412,30.987342,18.529412,30.987342
9,19767,1,22,00:00:22.273,967,18253.0,76.764706,33.569620,76.764706,33.569620


In [32]:
#events["extra"] = events["extra"].fillna({})
#events = events.fillna(0)
events.extra

0       {'tactics': {'formation': 41212, 'lineup': [{'...
1       {'tactics': {'formation': 42211, 'lineup': [{'...
2                                                      {}
3                                                      {}
4       {'pass': {'recipient': {'id': 15558, 'name': '...
5                                                      {}
6       {'pass': {'length': 55.226807, 'angle': -0.734...
7       {'pass': {'recipient': {'id': 10198, 'name': '...
8                                                      {}
9                                                      {}
10      {'pass': {'recipient': {'id': 18149, 'name': '...
11                                                     {}
12      {'pass': {'recipient': {'id': 10198, 'name': '...
13                                                     {}
14      {'pass': {'recipient': {'id': 15724, 'name': '...
15                                                     {}
16                                                     {}
17      {'pass

In [ ]:
#list(events.columns)
#set(events.type_name)
#events.location.apply(lambda x : x +1)
#events[["pass_end_location","shot_end_location"]]
columns
#events.location | events.pass_end_location
#set(events.type_name)